# Dunebuggy
---
A lightweight (unofficial) Python SDK for Dune.com

## Installation

```sh
pip install dunebuggy
```

## Getting started

### Retrieving a public query

To retrieve a query, all we'll need is the ```query_id``` for the public query we're interested in. In the below example we can take a look at the popular ["Custom NFT Floor Tracker" query by @smaroo](https://dune.com/queries/83579) (The ```query_id``` below can be found in the URL).

In [1]:
from dunebuggy import Dune

dune = Dune()
query = dune.fetch_query(83579)
query.df.head()


,Floor Price Ξ,Time Interval,Volume Ξ
0,0.0700,2021-06-30T00:00:00+00:00,436.250420
1,0.0670,2021-07-01T00:00:00+00:00,234.422398
2,0.0553,2021-07-02T00:00:00+00:00,126.205068
3,0.0530,2021-07-03T00:00:00+00:00,41.314672
4,0.0490,2021-07-04T00:00:00+00:00,40.311550


We can also take a look at some basic information about the returned query with ```query.info```

In [ ]:
query.info

Some queries in Dune are "parameterized", meaning that the author exposes certain variables for the user to enter custom values. The example query (83579) happens to be parameterized, we can verify this by inspecting ```query.parameters```

In [ ]:
query.parameters

If you'd like to run this query with your own custom parameters, all we'll need to do is take the parameters from from the initial query, change the values to what we want, and re-fetch the query. You can also create a fresh set of parameters by importing ```QueryParameter``` from ```dunebuggy.models.query``` and adding the values to the new object.

Below we are replacing the old NFT contract address param with a new one ([the contract address for BAYC](https://etherscan.io/address/0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d))

In [ ]:
old_params = query.parameters

# Replacing with contract address for BAYC
old_params[0].value = 'xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D'
new_params = old_params
print(new_params)
custom_query = dune.fetch_query(83579, parameters=new_params)

custom_query.info

Note that the ```result_id``` and ```job_id``` here are different, this is because we ran the query with our changed params

In [ ]:
custom_query.parameters

In [ ]:
custom_query.df.head()

### Creating a new query

Dunebuggy also allows you to create a new using an existing Dune.com account.You'll just need to pass in your username/password into the ```Dune``` object in order to login. After logging in, you should be able to retrieve your ```user_id```

In [ ]:
import os 

username = os.environ.get('DUNE_USERNAME')
password = os.environ.get('DUNE_PASSWORD')

dune = Dune(username=username, password=password)
dune.user_id

We'll now need to construct a Dune SQL query. We can do this in two ways. The first being just creating a raw string SQL query like below

In [2]:
query_string = "select * from ethereum.transactions\nLIMIT 100\n"

Or, if we wanna get fancy, we could use the fantastic PyPika library to construct one in an ORM style

In [3]:
from pypika import Database, Query

ethereum = Database('ethereum')
q = Query.from_(ethereum.transactions).select('*').limit(100)
query_string = q.get_sql(quote_char=None)
query_string

ModuleNotFoundError: No module named 'pypika'

Dune requires us to specify an integer code (```Id```) for each of their support blockchain datasets. The currently supported datasets are the following:

| Blockchain Dataset | Id |
|--------------------|----|
| ETHEREUM           | 4  |
| XDAI               | 6  |
| POLYGON            | 7  |
| OPTIMISM_1         | 8  |
| OPTIMISM_2         | 10 |
| BINANCE            | 9  |
| SOLANA             | 1  |

We can access these integer codes via the ```DatasetId``` enum. To create a query now, all we need to do is pass in a ```name```, ```query_string``` and ```dataset_id```

In [3]:
from dunebuggy.models.constants import DatasetId
created_query = dune.create_query("My Query's Name", query_string, DatasetId.ETHEREUM)                          
                       

In [4]:
created_query.df.head()

,access_list,block_hash,block_number,block_time,data,from,gas_limit,gas_price,gas_used,hash,index,max_fee_per_gas,max_priority_fee_per_gas,nonce,priority_fee_per_gas,success,to,type,value
0,None,\x887c665b0c52ccace092d817e984e2e828ef59079295...,47287,2015-08-07T08:50:01+00:00,None,\xdb312d1d6a2ccc64dd94a3892928bac82b4e8c15,21000,100000000000,21000,\xd3e6a2fc34066d20bb83020b1ee95b9dc7919fd242bd...,0,None,None,0,None,None,\x34bb6978c5a1ad68777ad388c6787df53903430c,None,1000000000000000000
1,None,\x4869e218b0a8f5784f16193ac66cbf35c4510ace0c9b...,48698,2015-08-07T15:29:53+00:00,None,\x48040276e9c17ddbe5c8d2976245dcd0235efa43,90000,57550496008,21000,\x8ba39f908731171fe96ee4e700e71d170ef8e651fac7...,0,None,None,0,None,None,\xd8d0549637b65d58e7fb6cbdd11530b399d1ddac,None,100000000000000000000
2,None,\xab9491b62b16bd928b281a83db82483584c22aeebc0d...,49051,2015-08-07T17:03:48+00:00,None,\x8686578c4f7c75246f548299d6ffdac3b67b5cd1,90000,57178423039,21000,\x57f8ba638903d6335e211eb470159587c73316788880...,0,None,None,0,None,None,\x87abffa6b80f712c852a9558120ba6611f0b5e46,None,45150000000000000000
3,None,\x1f9adc2190701ca3085b28252e4f1f467d980f763dad...,49174,2015-08-07T17:41:03+00:00,None,\x18e4ce47483b53040adbab35172c01ef64506e0c,90000,58589751415,21000,\xb8280da44f8d35011c3f431f7d1a82213477a4e742de...,2,None,None,0,None,None,\xfb26ae2d3621829472555fbd11bb2a324b7a5c57,None,10000000000000000000
4,None,\xf1f392fd197a149afe9f8843d7ba759d1a9f79d1ef62...,49938,2015-08-07T21:06:21+00:00,None,\xc6bf5b6558f2ee21f2e43d9ff9b5408a0cb89413,90000,71214529679,21000,\x538e1664c12c55287c98dc5dd248f60c642cbbbd7a18...,0,None,None,4,None,None,\x33a3f479f6c3e7f91128348490d1f7e8d2a0fab5,None,5000000000000000000


### Saving to CSV

To save a query to a CSV, we can take advantage of the ```to_csv``` method on our ```df```

In [ ]:
created_query.df.to_csv('my_test_data.csv')